# Credmark Modeling Framework Example for Jupyter notebook
## Introduction

version: 2022.6.3

In [ ]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext

from credmark.dto import *
from credmark.cmf.types import *

from credmark.cmf.engine.dev_models.console import get_dt, get_block, log_output

## Initialize

1. You could change the level of logging with `log_output()`.

In [ ]:
import logging
# Change output of logging to a file, and 
# Change the level to DEBUG when no level is specified.
# logging_output('../tmp/debug.log')

# Change level of logging from the default (WARNING) to
# either INFO (less information)
# or DEBUG(more information).
# Still output to stream.
log_output(log_level=logging.INFO)

2. Create context and some shortcuts for frequently used utilitis.

Update below parameters
- chain_id: 1 for ETH
- block_number: None or a specific number.
- chain_to_provider_url: web3 node (archive is preferred)
- api_url: None or a specific gateway server address
- console: True
- use_local_models: None, '*', or a comma-separated list.

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [ ]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
          'api_url': None,
          'run_id': None,
          'console': True,
          'use_local_models': None # Or, '*', or '-', or 'model_to_be_run_locally'
         }
context = EngineModelContext.create_context(**params)

ledger = context.ledger
run_model = context.run_model
models = context.models
block_number = context.block_number
chain_id = context.chain_id
web3 = context.web3
run_model_historical = context.historical.run_model_historical
run_model_historical_blocks = context.historical.run_model_historical_blocks

## Use Cmf

### 1. Basic utilities

In [ ]:
get_dt(2022, 5, 3)

In [ ]:
get_block(get_dt(2022, 5, 3))

In [ ]:
context.block_number

In [ ]:
context.chain_id

In [ ]:
context.web3

In [ ]:
context.web3.eth.get_balance('0xd3CdA913deB6f67967B99D67aCDFa1712C293601')

In [ ]:
context.web3.eth.get_block(context.block_number)['gasUsed']

### 2. Use models

1. Type `models.` and hit `TAB` key, a list of models will be pop up.
2. Select a model and type ? will show description and help, `models.token.info?`
3. Show the list of models wiht `dir(models)`, or `model_loader.loaded_model_versions()` with version information.
4. If you have updated any model in the folder on disk, run `model_loader.reload()` to refresh.

In [ ]:
dir(models)

In [ ]:
models.token.info?

In [ ]:
model_loader.reload()

## 3. Create and use types

### 3.1 Example - get price for USDC

In [ ]:
usdt = Token(address='0xdAC17F958D2ee523a2206206994597C13D831ec7')
print('USDT decimals:', usdt.decimals)
models.chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

In [ ]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

In [ ]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_registry(base=usdt)

### 3.1 Run a model

#### Run as of current block

In [ ]:
aave = Token(symbol='AAVE')

print(block_number,
      block_number.timestamp_datetime,
      'models:', context.models.price.quote(base=aave, return_type=Price).price,
      'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price).price)

#### Run as of past blocks

In [ ]:
context.block_number.timestamp_datetime

In [ ]:
for dt in [get_dt(2022, 5, 3), get_dt(2022, 5, 9), get_dt(2022, 1, 1)]:
    block = get_block(dt)
    print(block,
          block.timestamp_datetime,
          'models:', context.models(block).price.quote(base=aave, return_type=Price).price,
          'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price, block_number=block).price)

### 3.2 Travel between different blocks

Tavel is one-way only - can only travel to earlier/smaller block numbers, and not to the future/later blocks.

In [ ]:
context.block_number = get_block(get_dt(2022, 5, 3))
models.price.quote(base=aave, return_type=Price).price,

In [ ]:
# Below will gave any error as we wants to travel back
# context.block_number = get_block(get_dt(2022, 5, 30))
# models.chainlink.price_usd(input=aave, return_type=Price).price

Instead, create a new context. It will automatically become the current context.

In [ ]:
context_old = context.create_context(**params | {'block_number': get_block(get_dt(2022, 4, 3))})
context_old.block_number.timestamp_datetime, context.block_number.timestamp_datetime

If we would like to use previously defined context, run `context.set_current()` to set it back to the current context.

Otherwise, we may encounter error with running model for a block number later than the context.

In [ ]:
context.set_current()

### 4. Ledger

In [ ]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
             .to_dataframe())
df_ledger

In [ ]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
        .to_dataframe())
df_ledger